# importing library

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import scipy.cluster.hierarchy as sch
import plotly.figure_factory as ff

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.cluster import AgglomerativeClustering, AffinityPropagation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# importing data

In [3]:
data = pd.read_csv(r'https://github.com/ifood/ifood-data-business-analyst-test/blob/master/ml_project1_data.csv?raw=true', index_col=0)

# getting data information

In [4]:
data.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0


In [5]:
data.describe()

,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.000000,2216.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.0,2240.0,2240.000000
mean,1968.805804,52247.251354,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,37.525446,27.062946,44.021875,2.325000,4.084821,2.662054,5.790179,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
std,11.984069,25173.076661,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,54.628979,41.280498,52.167439,1.932238,2.778714,2.923101,3.250958,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274
min,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,1959.000000,35303.000000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,1.000000,9.000000,1.000000,2.000000,0.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,1970.000000,51381.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,8.000000,24.000000,2.000000,4.000000,2.000000,5.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,1977.000000,68522.000000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,33.000000,56.000000,3.000000,6.000000,4.000000,8.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
max,1996.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,263.000000,362.000000,15.000000,27.000000,28.000000,13.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.0,11.0,1.000000


In [6]:
data.Z_CostContact.value_counts()

3    2240
Name: Z_CostContact, dtype: int64

In [7]:
data.Z_Revenue.value_counts()

11    2240
Name: Z_Revenue, dtype: int64

## creating numerical features from categorical ones & others

In [8]:
data.Education.value_counts()

Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: Education, dtype: int64

In [9]:
def education_feature(value):
    '''Transform education to numerical values'''
    if value=='Basic': return 0
    elif value=='2n Cycle': return 1
    elif value=='Graduation': return 2
    elif value=='Master': return 3
    else: return 4

data['EDUCATION_NUM'] = data['Education'].apply(education_feature)
data.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,EDUCATION_NUM
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,2
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,2
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,2
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,2
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,4


In [48]:
data.Marital_Status.value_counts()

Married     864
Together    580
Single      487
Divorced    232
Widow        77
Name: Marital_Status, dtype: int64

In [11]:
def marital_feature(value):
    '''
    Transform marital status in numerical values
    Different status are returned as single
    '''
    if value=='Married': return 4
    elif value=='Together': return 3
    elif value=='Widow': return 2
    elif value=='Divorced': return 1
    else: return 0

data['MARITAL_NUM'] = data['Marital_Status'].apply(marital_feature)
data.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,EDUCATION_NUM,MARITAL_NUM
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,2,0
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,2,0
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,2,3
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,2,3
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,4,4


In [44]:
data.replace({
    'Alone': 'Single',
'Absurd': 'Single',
'YOLO': 'Single'
}, inplace=True)

In [12]:
# get year of clients loyalty 
data['YEARS_CLIENT'] = dt.datetime.today().year - pd.to_datetime(data['Dt_Customer']).dt.year
data.head()


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,EDUCATION_NUM,MARITAL_NUM,YEARS_CLIENT
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,2,0,11
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,2,0,9
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,2,3,10
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,2,3,9
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,4,4,9


In [13]:
# get age of clients 
data['AGE'] = dt.datetime.today().year - data['Year_Birth']
data.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,EDUCATION_NUM,MARITAL_NUM,YEARS_CLIENT,AGE
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,2,0,11,66
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,2,0,9,69
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,2,3,10,58
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,2,3,9,39
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,4,4,9,42


In [56]:
fig = px.histogram(data, x="AGE")
fig.show()

In [54]:
data['AGE'].describe()

count    2240.000000
mean       54.194196
std        11.984069
min        27.000000
25%        46.000000
50%        53.000000
75%        64.000000
max       130.000000
Name: AGE, dtype: float64

In [ ]:
data

In [ ]:
def age_range(value):
    '''Transform age to range'''
    if value>: return 0
    elif value=='2n Cycle': return 1
    elif value=='Graduation': return 2
    elif value=='Master': return 3
    else: return 4

data['AGE'] = data['Education'].apply(education_feature)
data.head()

In [26]:
# setting all accepted clients (except response)
data['TOTAL_ACCEPTED'] = data[['AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','AcceptedCmp1','AcceptedCmp2']].sum(axis=1)#	Response


data['TOTAL_ACCEPTED_RESPONSE'] = data[['AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','AcceptedCmp1','AcceptedCmp2', 'Response']].sum(axis=1)#	Response
data

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,EDUCATION_NUM,MARITAL_NUM,YEARS_CLIENT,AGE,TOTAL_ACCEPTED,TOTAL_ACCEPTED_RESONSE,TOTAL_ACCEPTED_RESPONSE
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,2,0,11,66,0,1,1
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,2,0,9,69,0,0,0
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,2,3,10,58,0,0,0
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,2,3,9,39,0,0,0
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,4,4,9,42,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,43,182,42,118,247,2,9,3,4,5,0,0,0,0,0,0,3,11,0,2,4,10,56,0,0,0
4001,1946,PhD,Together,64014.0,2,1,2014-06-10,56,406,0,30,0,0,8,7,8,2,5,7,0,0,0,1,0,0,3,11,0,4,3,9,77,1,1,1
7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,48,217,32,12,24,1,2,3,13,6,0,1,0,0,0,0,3,11,0,2,1,9,42,1,1,1


In [15]:
data.TOTAL_ACCEPTED.value_counts()

0    1777
1     325
2      83
3      44
4      11
Name: TOTAL_ACCEPTED, dtype: int64

In [16]:
data.Response.value_counts()

0    1906
1     334
Name: Response, dtype: int64

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 5524 to 9405
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year_Birth           2240 non-null   int64  
 1   Education            2240 non-null   object 
 2   Marital_Status       2240 non-null   object 
 3   Income               2216 non-null   float64
 4   Kidhome              2240 non-null   int64  
 5   Teenhome             2240 non-null   int64  
 6   Dt_Customer          2240 non-null   object 
 7   Recency              2240 non-null   int64  
 8   MntWines             2240 non-null   int64  
 9   MntFruits            2240 non-null   int64  
 10  MntMeatProducts      2240 non-null   int64  
 11  MntFishProducts      2240 non-null   int64  
 12  MntSweetProducts     2240 non-null   int64  
 13  MntGoldProds         2240 non-null   int64  
 14  NumDealsPurchases    2240 non-null   int64  
 15  NumWebPurchases      2240 non-null 

## data undertanding

In [72]:
response_gp = data.groupby('Response').sum().reset_index(drop=False)
fig = px.bar(response_gp, y=['Kidhome',
       'Teenhome', 'Complain', 'TOTAL_ACCEPTED'], x='Response', barmode="group")
fig.show()

C:\Users\giane.pessoa\AppData\Local\Temp\ipykernel_29136\1378149313.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [ ]:
education = data.groupby('EDUCATION_NUM').agg(
    {'Response': 'sum', 'Income': 'sum', 'AGE': 'count'}
).reset_index(drop=False)
# education
fig = go.Figure([go.Bar(x=education['EDUCATION_NUM'], y=education['Response'])])
fig.show()
fig = go.Figure([go.Bar(x=education['EDUCATION_NUM'], y=education['AGE'])])
fig.show()

In [49]:
aceepted_cons = data.loc[(data['TOTAL_ACCEPTED']>0) & (data['Response']>0)] 
aceepted_cons.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,EDUCATION_NUM,MARITAL_NUM,YEARS_CLIENT,AGE,TOTAL_ACCEPTED,TOTAL_ACCEPTED_RESONSE,TOTAL_ACCEPTED_RESPONSE
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2114,1946,PhD,Single,82800.0,0,0,2012-11-24,23,1006,22,115,59,68,45,1,7,6,12,3,0,0,1,1,0,0,3,11,1,4,0,11,77,2,3,3
2968,1943,PhD,Divorced,48948.0,0,0,2013-02-01,53,437,8,206,160,49,42,2,7,10,5,6,1,0,0,0,0,0,3,11,1,4,1,10,80,1,2,2
2225,1977,Graduation,Divorced,82582.0,0,0,2014-06-07,54,510,120,550,156,40,241,1,4,9,7,1,1,0,0,1,0,0,3,11,1,2,1,9,46,2,3,3
6260,1955,Master,Together,82384.0,0,0,2012-11-19,55,984,51,432,180,120,190,1,3,10,13,1,0,0,1,0,0,0,3,11,1,3,3,11,68,1,2,2
6853,1982,Master,Single,75777.0,0,0,2013-07-04,12,712,26,538,69,13,80,1,3,6,11,1,0,1,1,0,0,0,3,11,1,3,0,10,41,2,3,3


In [39]:
# education = aceepted_cons.groupby('Education').count().reset_index(drop=False)
# education
fig = px.bar(education, x='Education', y='TOTAL_ACCEPTED_RESPONSE' , barmode="group", title='Distribuição de clientes por nível educacional', width=700)
fig.show()

In [50]:
marital = aceepted_cons.groupby('Marital_Status').sum()['TOTAL_ACCEPTED_RESPONSE'].reset_index(drop=False)
# marital
fig = px.bar(marital, x='Marital_Status', y='TOTAL_ACCEPTED_RESPONSE' , barmode="group", title='Distribuição de clientes por estado civil', width=700)
fig.show()

C:\Users\giane.pessoa\AppData\Local\Temp\ipykernel_22272\1247057820.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [57]:
fig = px.histogram(aceepted_cons, x="AGE")
fig.show()

In [60]:
# MntWines	MntFruits	MntMeatProducts	MntFishProducts	MntSweetProducts	MntGoldProds	NumDealsPurchases	NumWebPurchases	NumCatalogPurchases	NumStorePurchases	NumWebVisitsMonth
fig = px.histogram(aceepted_cons, x="MntWines")
fig.show()

fig = px.histogram(aceepted_cons, x="MntFruits")
fig.show()

fig = px.histogram(aceepted_cons, x="MntFishProducts")
fig.show()

fig = px.histogram(aceepted_cons, x="MntGoldProds")
fig.show()

fig = px.histogram(aceepted_cons, x="NumWebPurchases")
fig.show()

fig = px.histogram(aceepted_cons, x="NumWebVisitsMonth")
fig.show()

fig = px.histogram(aceepted_cons, x="NumDealsPurchases")
fig.show()

In [52]:

child_home = aceepted_cons.groupby(['Kidhome','Teenhome']).sum()['TOTAL_ACCEPTED_RESPONSE'].reset_index(drop=False)
# marital
fig = px.bar(child_home, x='Kidhome', y='TOTAL_ACCEPTED_RESPONSE' , barmode="group", title='Distribuição de clientes por quantidade de crianças em casa', width=700)
fig.show()
fig = px.bar(child_home, x='Teenhome', y='TOTAL_ACCEPTED_RESPONSE' , barmode="group", title='Distribuição de clientes por quantidade de crianças em casa', width=700)
fig.show()

C:\Users\giane.pessoa\AppData\Local\Temp\ipykernel_22272\3224512293.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



## getting insights over correlation

In [17]:
z = data.corr()

fig = px.imshow(z, text_auto=True)
fig.show()

C:\Users\giane.pessoa\AppData\Local\Temp\ipykernel_25692\2028650979.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  z = data.corr()


In [18]:
data.columns

Index(['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response',
       'EDUCATION_NUM', 'MARITAL_NUM', 'YEARS_CLIENT', 'AGE',
       'TOTAL_ACCEPTED'],
      dtype='object')

In [19]:
data[['MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts'
    , 'Income']].corr()

,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,Income
MntWines,1.000000,0.389637,0.562667,0.399753,0.386581,0.578650
MntFruits,0.389637,1.000000,0.543105,0.594804,0.567164,0.430842
MntMeatProducts,0.562667,0.543105,1.000000,0.568402,0.523846,0.584633
MntFishProducts,0.399753,0.594804,0.568402,1.000000,0.579870,0.438871
MntSweetProducts,0.386581,0.567164,0.523846,0.579870,1.000000,0.440744
Income,0.578650,0.430842,0.584633,0.438871,0.440744,1.000000


In [20]:
data[['NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases','Income']].corr()

,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,Income
NumDealsPurchases,1.000000,0.234185,-0.008617,0.068879,-0.083101
NumWebPurchases,0.234185,1.000000,0.378376,0.502713,0.387878
NumCatalogPurchases,-0.008617,0.378376,1.000000,0.518738,0.589162
NumStorePurchases,0.068879,0.502713,0.518738,1.000000,0.529362
Income,-0.083101,0.387878,0.589162,0.529362,1.000000


com um "zoom" em algumas correlações é possível observar algumas similaridades

In [21]:
data[['MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts'
    , 'Teenhome', 'Kidhome']].corr()

,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,Teenhome,Kidhome
MntWines,1.000000,0.389637,0.562667,0.399753,0.386581,0.004846,-0.496297
MntFruits,0.389637,1.000000,0.543105,0.594804,0.567164,-0.176764,-0.372581
MntMeatProducts,0.562667,0.543105,1.000000,0.568402,0.523846,-0.261160,-0.437129
MntFishProducts,0.399753,0.594804,0.568402,1.000000,0.579870,-0.204187,-0.387644
MntSweetProducts,0.386581,0.567164,0.523846,0.579870,1.000000,-0.162475,-0.370673
Teenhome,0.004846,-0.176764,-0.261160,-0.204187,-0.162475,1.000000,-0.036133
Kidhome,-0.496297,-0.372581,-0.437129,-0.387644,-0.370673,-0.036133,1.000000


In [22]:
data.corr()['Kidhome']

C:\Users\giane.pessoa\AppData\Local\Temp\ipykernel_25692\786773099.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Year_Birth             0.230176
Income                -0.428669
Kidhome                1.000000
Teenhome              -0.036133
Recency                0.008827
MntWines              -0.496297
MntFruits             -0.372581
MntMeatProducts       -0.437129
MntFishProducts       -0.387644
MntSweetProducts      -0.370673
MntGoldProds          -0.349595
NumDealsPurchases      0.221798
NumWebPurchases       -0.361647
NumCatalogPurchases   -0.502237
NumStorePurchases     -0.499683
NumWebVisitsMonth      0.447846
AcceptedCmp3           0.014674
AcceptedCmp4          -0.161600
AcceptedCmp5          -0.205634
AcceptedCmp1          -0.172339
AcceptedCmp2          -0.081716
Complain               0.040207
Z_CostContact               NaN
Z_Revenue                   NaN
Response              -0.080008
EDUCATION_NUM         -0.053660
MARITAL_NUM            0.005990
YEARS_CLIENT          -0.053339
AGE                   -0.230176
TOTAL_ACCEPTED        -0.211892
Name: Kidhome, dtype: float64

quando o cliente tem filhos pequenos, o consumo desacelera

In [23]:
data.corr()['NumWebVisitsMonth']

C:\Users\giane.pessoa\AppData\Local\Temp\ipykernel_25692\1414359402.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Year_Birth             0.121139
Income                -0.553088
Kidhome                0.447846
Teenhome               0.134884
Recency               -0.021445
MntWines              -0.320653
MntFruits             -0.418383
MntMeatProducts       -0.539470
MntFishProducts       -0.446003
MntSweetProducts      -0.423294
MntGoldProds          -0.250719
NumDealsPurchases      0.347633
NumWebPurchases       -0.055846
NumCatalogPurchases   -0.520364
NumStorePurchases     -0.428473
NumWebVisitsMonth      1.000000
AcceptedCmp3           0.061211
AcceptedCmp4          -0.032126
AcceptedCmp5          -0.278113
AcceptedCmp1          -0.192502
AcceptedCmp2          -0.007196
Complain               0.019769
Z_CostContact               NaN
Z_Revenue                   NaN
Response              -0.003987
EDUCATION_NUM         -0.055404
MARITAL_NUM            0.004763
YEARS_CLIENT           0.249489
AGE                   -0.121139
TOTAL_ACCEPTED        -0.166345
Name: NumWebVisitsMonth, dtype: float64

In [24]:
data.corr()['Income']

C:\Users\giane.pessoa\AppData\Local\Temp\ipykernel_25692\74929296.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Year_Birth            -0.161791
Income                 1.000000
Kidhome               -0.428669
Teenhome               0.019133
Recency               -0.003970
MntWines               0.578650
MntFruits              0.430842
MntMeatProducts        0.584633
MntFishProducts        0.438871
MntSweetProducts       0.440744
MntGoldProds           0.325916
NumDealsPurchases     -0.083101
NumWebPurchases        0.387878
NumCatalogPurchases    0.589162
NumStorePurchases      0.529362
NumWebVisitsMonth     -0.553088
AcceptedCmp3          -0.016174
AcceptedCmp4           0.184400
AcceptedCmp5           0.335943
AcceptedCmp1           0.276820
AcceptedCmp2           0.087545
Complain              -0.027225
Z_CostContact               NaN
Z_Revenue                   NaN
Response               0.133047
EDUCATION_NUM          0.149338
MARITAL_NUM            0.007957
YEARS_CLIENT          -0.022451
AGE                    0.161791
TOTAL_ACCEPTED         0.308381
Name: Income, dtype: float64

## looking for outliers

In [25]:

fig = px.box(data, y="Income")
fig.show()

In [26]:
data.loc[data['Income'] > 115000]

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,EDUCATION_NUM,MARITAL_NUM,YEARS_CLIENT,AGE,TOTAL_ACCEPTED
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8475,1973,PhD,Married,157243.0,0,1,2014-03-01,98,20,2,1582,1,2,1,15,0,22,0,0,0,0,0,0,0,0,3,11,0,4,4,9,50,0
1503,1976,PhD,Together,162397.0,1,1,2013-06-03,31,85,1,16,2,1,2,0,0,0,1,1,0,0,0,0,0,0,3,11,0,4,3,10,47,0
5555,1975,Graduation,Divorced,153924.0,0,0,2014-02-07,81,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,3,11,0,2,1,9,48,0
1501,1982,PhD,Married,160803.0,0,0,2012-08-04,21,55,16,1622,17,3,4,15,0,28,1,0,0,0,0,0,0,0,3,11,0,4,4,11,41,0
5336,1971,Master,Together,157733.0,1,0,2013-06-04,37,39,1,9,2,0,8,0,1,0,1,1,0,0,0,0,0,0,3,11,0,3,3,10,52,0
4931,1977,Graduation,Together,157146.0,0,0,2013-04-29,13,1,0,1725,2,1,1,0,0,28,0,1,0,0,0,0,0,0,3,11,0,2,3,10,46,0
11181,1949,PhD,Married,156924.0,0,0,2013-08-29,85,2,1,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,3,11,0,4,4,10,74,0
9432,1977,Graduation,Together,666666.0,1,0,2013-06-02,23,9,14,18,8,1,12,4,3,1,3,6,0,0,0,0,0,0,3,11,0,2,3,10,46,0


Há apenas 8 registros outliers em Income. Por ser um indicador bastante relevante com correlação interessante com vários outros indicadores, serão filtrados esses outliers.
É possível fazer uma análise da base completa, a fim de verificar se eles são outliers na base completa de clientes e talvez modelar de acordo com clusters, mas nesse momento, por simplificação, serão retirados.

In [22]:
# filtering outliers and 
df_data = data.loc[data['Income'] < 115000]

fig = px.box(df_data, y="Income")
fig.show()

In [28]:
fig = px.box(df_data, y=['MntWines', 'MntFruits',
       'MntFishProducts', 'MntSweetProducts'])
fig.show()

In [29]:
fig = px.box(df_data, y=['NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases'])
fig.show()

       

In [37]:
hist_data = [df_data['EDUCATION_NUM'], df_data['MARITAL_NUM']]
group_labels = ['Education', 'Marital Status']

fig = ff.create_distplot(hist_data, group_labels, bin_size=[5, 10, 8])
fig.update_layout(title_text='Age, Income and Score distribution')
fig.show()

In [39]:
fig = px.scatter(df_data, x="EDUCATION_NUM", y = "YEARS_CLIENT",size='TOTAL_ACCEPTED', color="Response")
fig.show()

# obtaining clusters

In [31]:
df_data.columns

Index(['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response',
       'EDUCATION_NUM', 'MARITAL_NUM', 'YEARS_CLIENT', 'AGE',
       'TOTAL_ACCEPTED'],
      dtype='object')

In [23]:
x = df_data[['Income', 'Kidhome',
       'Teenhome', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 
       'EDUCATION_NUM', 'MARITAL_NUM', 'YEARS_CLIENT', 'AGE']]
y = df_data['Response']

In [33]:
wcss = []
for i in range(1, 12):
    kmeans = KMeans(n_clusters = i, init = "k-means++", max_iter = 500, n_init = 10, random_state = 47)
    kmeans.fit(x)
    wcss.append(kmeans.inertia_)
    
fig = go.Figure(data = go.Scatter(x = [1,2,3,4,5,6,7,8,9,10, 11, 12], y = wcss))


fig.update_layout(title='WCSS vs. Cluster',
                   xaxis_title='Clusters',
                   yaxis_title='WCSS')
fig.show()

In [34]:
kmeans = KMeans(n_clusters = 3, init="k-means++", max_iter = 500, n_init = 10, random_state = 47)
identified_clusters = kmeans.fit_predict(x)

data_with_clusters = df_data.copy()
data_with_clusters['Cluster'] = identified_clusters

In [24]:
#Normalize the values
X = StandardScaler().fit_transform(x)

wcss = []
for i in range(1, 12):
    kmeans = KMeans(n_clusters = i, init = "k-means++", max_iter = 500, n_init = 10, random_state = 47)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    
fig = go.Figure(data = go.Scatter(x = [1,2,3,4,5,6,7,8,9,10, 11, 12], y = wcss))


fig.update_layout(title='WCSS vs. Cluster',
                   xaxis_title='Clusters',
                   yaxis_title='WCSS')
fig.show()

In [36]:
kmeans = KMeans(n_clusters = 7, init="k-means++", max_iter = 500, n_init = 10, random_state = 47)
identified_clusters = kmeans.fit_predict(x)

data_with_clusters_normal = df_data.copy()
data_with_clusters_normal['Cluster'] = identified_clusters
data_with_clusters_normal.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,EDUCATION_NUM,MARITAL_NUM,YEARS_CLIENT,AGE,TOTAL_ACCEPTED,Cluster
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,2,0,11,66,0,3
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,2,0,9,69,0,2
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,2,3,10,58,0,1
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,2,3,9,39,0,0
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,4,4,9,42,0,3


In [37]:
data_with_clusters.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,EDUCATION_NUM,MARITAL_NUM,YEARS_CLIENT,AGE,TOTAL_ACCEPTED,Cluster
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,2,0,11,66,0,2
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,2,0,9,69,0,2
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,2,3,10,58,0,1
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,2,3,9,39,0,0
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,4,4,9,42,0,2


In [38]:
# fig = px.scatter(data_with_clusters, x="", y = "Response",size='TOTAL_ACCEPTED', color="Cluster")
# fig.show()

fig = px.scatter_matrix(data_with_clusters,
    dimensions=['MntFruits', 'MntGoldProds', 'NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases'],#'Recency', 'EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACEPTED'],
    color="Cluster",
    title="Scatter matrix",
    labels={col:col.replace('_', ' ') for col in data_with_clusters.columns}) # remove underscore

fig.update_traces(diagonal_visible=False)
fig.show()

c:\Users\giane.pessoa\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [39]:
fig = px.scatter_matrix(data_with_clusters,
    dimensions=['EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACCEPTED'],#'Recency', 'EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACEPTED'],
    color="Cluster",
    title="Scatter matrix",
    labels={col:col.replace('_', ' ') for col in data_with_clusters.columns}) # remove underscore

fig.update_traces(diagonal_visible=False)
fig.show()

c:\Users\giane.pessoa\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [40]:
fig = px.scatter(data_with_clusters,
    x='NumWebPurchases',y='NumCatalogPurchases',#'Recency', 'EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACEPTED'],
    color="Cluster",)
    # title="Scatter matrix",
    # labels={col:col.replace('_', ' ') for col in data_with_clusters.columns}) # remove underscore

# fig.update_traces(diagonal_visible=False)
fig.show()

In [41]:

fig = px.scatter_matrix(data_with_clusters_normal,
    dimensions=['MntFruits', 'MntGoldProds', 'NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases'],#'Recency', 'EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACEPTED'],
    color="Cluster",
    title="Scatter matrix",
    labels={col:col.replace('_', ' ') for col in data_with_clusters.columns}) # remove underscore

fig.update_traces(diagonal_visible=False)
fig.show()

c:\Users\giane.pessoa\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [42]:
fig = px.scatter_matrix(data_with_clusters_normal,
    dimensions=['EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACCEPTED'],#'Recency', 'EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACEPTED'],
    color="Cluster",
    title="Scatter matrix",
    labels={col:col.replace('_', ' ') for col in data_with_clusters.columns}) # remove underscore

fig.update_traces(diagonal_visible=False)
fig.show()

c:\Users\giane.pessoa\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [43]:
fig = px.scatter(data_with_clusters_normal,
    x='EDUCATION_NUM',y='TOTAL_ACCEPTED',#'Recency', 'EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACEPTED'],
    color="Cluster",)
    # title="Scatter matrix",
    # labels={col:col.replace('_', ' ') for col in data_with_clusters.columns}) # remove underscore

# fig.update_traces(diagonal_visible=False)
fig.show()

## hierarchical clustering 

In [25]:
#Normalize the values
X = StandardScaler().fit_transform(x)
db = DBSCAN(eps=0.4, min_samples=20)
db.fit(X)

data_with_clusters_dbscan = df_data.copy()
data_with_clusters_dbscan['Cluster'] = identified_clusters
data_with_clusters_dbscan.Cluster.value_counts()

NameError: name 'identified_clusters' is not defined

In [45]:

fig = px.imshow(data_with_clusters_dbscan.corr(), text_auto=True)
fig.show()

C:\Users\giane.pessoa\AppData\Local\Temp\ipykernel_25692\145105924.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [46]:
fig = px.scatter_matrix(data_with_clusters_dbscan,
    dimensions=['EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACCEPTED'],#'Recency', 'EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACEPTED'],
    color="Cluster",
    title="Scatter matrix",
    labels={col:col.replace('_', ' ') for col in data_with_clusters.columns}) # remove underscore

fig.update_traces(diagonal_visible=False)
fig.show()

c:\Users\giane.pessoa\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [47]:
fig = px.scatter_matrix(data_with_clusters_normal,
    dimensions=['MntFruits', 'MntGoldProds', 'NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases'],#'Recency', 'EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACEPTED'],
    color="Cluster",
    title="Scatter matrix",
    labels={col:col.replace('_', ' ') for col in data_with_clusters.columns}) # remove underscore

fig.update_traces(diagonal_visible=False)
fig.show()

c:\Users\giane.pessoa\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [55]:
fig = px.scatter(data_with_clusters_normal,
    x='NumDealsPurchases',y='Response',#'EDUCATION_NUM','MARITAL_NUM','YEARS_CLIENT','AGE','TOTAL_ACCEPTED'
    color="Cluster",)
    # title="Scatter matrix",
    # labels={col:col.replace('_', ' ') for col in data_with_clusters.columns}) # remove underscore

# fig.update_traces(diagonal_visible=False)
fig.show()

In [49]:
# fig = ff.create_dendrogram(x,
#                            linkagefun = lambda x: sch.linkage(x, "ward"),)

# # Ward minimizes the variance of the points inside a cluster.

# fig.update_layout(title = 'Hierarchical Clustering', xaxis_title='Customers',
#                    yaxis_title='Euclidean Distance', width=700, height=700)

# fig.show()

# classification model

In [26]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


clf=RandomForestClassifier()#n_estimators=100

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8808446455505279


In [27]:
importance = pd.DataFrame(x.columns)
importance['score'] = clf.feature_importances_
importance.sort_values(by='score', ascending=False, inplace=True)
importance.reset_index(drop=True, inplace=True)

In [28]:
importance.iloc[:9][0].values

array(['Recency', 'MntMeatProducts', 'Income', 'MntWines', 'MntGoldProds',
       'AGE', 'MntSweetProducts', 'AcceptedCmp1', 'NumStorePurchases'],
      dtype=object)

In [32]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(x[['Recency', 'MntMeatProducts', 'Income', 'MntWines', 'MntGoldProds',
       'AGE', 'MntSweetProducts', 'AcceptedCmp1', 'NumStorePurchases']], y, test_size=0.3)

clf=RandomForestClassifier()#n_estimators=100

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

Accuracy: 0.8778280542986425
              precision    recall  f1-score   support

           0       0.90      0.97      0.93       572
           1       0.60      0.33      0.43        91

    accuracy                           0.88       663
   macro avg       0.75      0.65      0.68       663
weighted avg       0.86      0.88      0.86       663



In [30]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

Accuracy: 0.8778280542986425
              precision    recall  f1-score   support

           0       0.91      0.96      0.93       567
           1       0.62      0.41      0.49        96

    accuracy                           0.88       663
   macro avg       0.76      0.68      0.71       663
weighted avg       0.86      0.88      0.87       663



c:\Users\giane.pessoa\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



In [33]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
for train, test in kf.split(X):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.95      0.93       572
           1       0.53      0.33      0.41        91

    accuracy                           0.87       663
   macro avg       0.71      0.64      0.67       663
weighted avg       0.85      0.87      0.85       663

              precision    recall  f1-score   support

           0       0.90      0.95      0.93       572
           1       0.53      0.33      0.41        91

    accuracy                           0.87       663
   macro avg       0.71      0.64      0.67       663
weighted avg       0.85      0.87      0.85       663

              precision    recall  f1-score   support

           0       0.90      0.95      0.93       572
           1       0.53      0.33      0.41        91

    accuracy                           0.87       663
   macro avg       0.71      0.64      0.67       663
weighted avg       0.85      0.87      0.85       663

              preci

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.1)
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       572
           1       0.58      0.33      0.42        91

    accuracy                           0.87       663
   macro avg       0.74      0.65      0.67       663
weighted avg       0.86      0.87      0.86       663

